In [1]:
%tensorflow_version 1.x   #Using tensorflow 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':      #use GPU for fast process
  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x   #Using tensorflow 1.x`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
print(tf.__version__)

1.15.2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import requests  
file_url = "http://nlp.stanford.edu/data/glove.6B.zip"          #Download glove here is the link
    
r = requests.get(file_url, stream = True)  
  
with open("glove.6B.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block) 

In [5]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [6]:
cd '/content/go'      #downloaded in server but not in my drive it is fast to access

[Errno 2] No such file or directory: '/content/go #downloaded in server but not in my drive it is fast to access'
/content


In [7]:
!unzip  '/content/glove.6B.zip' -d go

Archive:  /content/glove.6B.zip
  inflating: go/glove.6B.50d.txt     
  inflating: go/glove.6B.100d.txt    
  inflating: go/glove.6B.200d.txt    
  inflating: go/glove.6B.300d.txt    


In [8]:
'''import numpy as np
np.random.seed(42)
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D,GlobalAveragePooling1D
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')'''
import numpy as np
import pandas as pd
import re
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [206]:
EMBEDDING_FILE = '/content/go/glove.6B.300d.txt'      
train = pd.read_csv('/content/drive/My Drive/Hate Speech/english_dataset.tsv', sep = '\t', encoding="latin-1")
test = pd.read_csv('/content/drive/My Drive/Hate Speech/hasoc2019_en_test-2919.tsv', sep = '\t', encoding="latin-1")

In [207]:
train=train[train.task_2!="NONE"]       #drop none
test=test[test.task_2!="NONE"]
train=train.reset_index(drop=True)    #reset index
test=test.reset_index(drop=True)
X_train = train["text"]
y_train = train["task_2"]       #For task 2
X_test = test["text"]
y_test = test["task_2"]

In [208]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [209]:
#Remove stopwords from the text
stop_words = set(stopwords.words('english')) 
ix=0
for i in X_train:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_train[ix]=X_train[ix]+(w) 
  ix=ix+1
ix=0
for i in X_test:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_test[ix]= X_test[ix]+(w) 
  ix=ix+1

In [210]:
#remove punctuations
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

In [211]:
ix=0
for i in X_train:
  X_train[ix]=clean_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_text(i)
  ix=ix+1

In [212]:
#clean the numbers
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [213]:
ix=0
for i in X_train:
  X_train[ix]=clean_numbers(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_numbers(i)
  ix=ix+1

In [214]:
#remove contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


In [215]:
ix=0
for i in X_train:
  X_train[ix]=replace_contractions(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=replace_contractions(i)
  ix=ix+1

In [216]:
#Stemming
'''Stemming is the process of converting words to their base forms using crude Heuristic rules.
   For example, one rule could be to remove ’s’ from the end of any word, so that ‘cats’ becomes ‘cat’. 
   or another rule could be to replace ‘ies’ with ‘i’ so that ‘ponies becomes ‘poni’.'''
from nltk.stem import  SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
def stem_text(text):
    tokenizer = ToktokTokenizer()
    stemmer = SnowballStemmer('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

In [217]:
ix=0
for i in X_train:
  X_train[ix]=stem_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=stem_text(i)
  ix=ix+1

In [218]:
type(X_train)

pandas.core.series.Series

In [219]:
#After removing or cleaning the text
max_features = 11051   #No. of different words in total text 5852, 2261
maxlen = 70           #Padding length
embed_size = 300      #No. of dimensions of glove

threshold = 0.35

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [220]:
y_train = y_train.map({'HATE':0, 'OFFN': 1,'PRFN':2})    #Map accordingly 
y_test = y_test.map({'HATE':0, 'OFFN': 1,'PRFN':2})

In [221]:
y_train=keras.utils.to_categorical(y_train, num_classes=3)  #one hot encoding
y_test=keras.utils.to_categorical(y_test, num_classes=3)


In [222]:
y=[]
for i in y_train:
 y.append(np.argmax(i))

In [223]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [224]:
num_filters = 42
filter_sizes = [1,2,3,5]  #ngrams

In [225]:
#CNN

def get_model():    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
#    x = SpatialDropout1D(0.4)(x)
    x = Reshape((maxlen, embed_size, 1))(x)
    
    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), 
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    
    maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)
    maxpool_3 = MaxPool2D(pool_size=(maxlen - filter_sizes[3] + 1, 1))(conv_3)
        
    z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])   
    z = Flatten()(z)
    z = Dropout(0.1)(z)
        
    outp = Dense(3, activation="sigmoid")(z)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()

In [226]:
#model of bi-lstm
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    CuDNNLSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(3, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [227]:
#model of lstm 
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_slstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    LSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(3, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [228]:
#model of gru 
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_gru_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    GRU is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(3, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [229]:
#model of Simple RNN 
from keras.layers import Dense, Input, SimpleRNN, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_rnn_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    SimpleRNN is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(SimpleRNN(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(3, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [230]:
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 3s - loss: 1.0525 - accuracy: 0.4420 - val_loss: 0.9786 - val_accuracy: 0.5263
Epoch 2/10
 - 0s - loss: 0.9384 - accuracy: 0.5137 - val_loss: 0.9560 - val_accuracy: 0.5439
Epoch 3/10
 - 0s - loss: 0.8583 - accuracy: 0.5449 - val_loss: 0.9467 - val_accuracy: 0.5614
Epoch 4/10
 - 0s - loss: 0.7646 - accuracy: 0.6143 - val_loss: 0.9327 - val_accuracy: 0.5439
Epoch 5/10
 - 0s - loss: 0.6508 - accuracy: 0.7788 - val_loss: 0.9243 - val_accuracy: 0.5614
Epoch 6/10
 - 0s - loss: 0.5433 - accuracy: 0.8011 - val_loss: 0.9281 - val_accuracy: 0.5614
Epoch 7/10
 - 0s - loss: 0.4373 - accuracy: 0.9073 - val_loss: 0.9163 - val_accuracy: 0.5965
Epoch 8/10
 - 0s - loss: 0.3631 - accuracy: 0.9287 - val_loss: 0.9011 - val_accuracy: 0.6053
Epoch 9/10
 - 0s - loss: 0.2900 - accuracy: 0.9595 - val_loss: 0.9157 - val_accuracy: 0.6140
Epoch 10/10
 - 0s - loss: 0.2297 - accuracy: 0.9776 - val_loss: 0.9071 - val_accuracy: 0.6140


In [231]:
print(tf.__version__)

1.15.2


In [232]:
ls_model = model_lstm_atten(embedding_matrix) #model of bi-lstm with attention

In [233]:
sls_model = model_slstm_atten(embedding_matrix) #model of lstm with attention

In [234]:
gru_model = model_gru_atten(embedding_matrix) #model of gru with attention

In [235]:
rnn_model = model_rnn_atten(embedding_matrix) #model of rnn with attention

In [236]:
batch_size = 256
epochs = 10

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95,
                                              random_state=42) #split this into train and validation

In [237]:
ls_hist = ls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train bi-lstm with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 3s - loss: 1.0574 - accuracy: 0.4690 - val_loss: 0.9885 - val_accuracy: 0.5263
Epoch 2/10
 - 0s - loss: 1.0068 - accuracy: 0.5044 - val_loss: 0.9743 - val_accuracy: 0.5263
Epoch 3/10
 - 0s - loss: 0.9544 - accuracy: 0.5207 - val_loss: 0.9541 - val_accuracy: 0.5702
Epoch 4/10
 - 0s - loss: 0.8736 - accuracy: 0.5855 - val_loss: 0.9586 - val_accuracy: 0.5877
Epoch 5/10
 - 0s - loss: 0.7681 - accuracy: 0.6577 - val_loss: 0.9471 - val_accuracy: 0.6053
Epoch 6/10
 - 0s - loss: 0.6364 - accuracy: 0.7098 - val_loss: 0.9513 - val_accuracy: 0.6140
Epoch 7/10
 - 0s - loss: 0.5376 - accuracy: 0.7611 - val_loss: 1.0260 - val_accuracy: 0.6316
Epoch 8/10
 - 0s - loss: 0.4311 - accuracy: 0.8179 - val_loss: 1.1046 - val_accuracy: 0.5877
Epoch 9/10
 - 0s - loss: 0.3388 - accuracy: 0.8659 - val_loss: 1.2345 - val_accuracy: 0.5702
Epoch 10/10
 - 0s - loss: 0.2604 - accuracy: 0.9101 - val_loss: 1.2897 - val_accuracy: 0.5439


In [238]:
sls_hist = sls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)    #Train lstm with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 4s - loss: 1.0512 - accuracy: 0.4835 - val_loss: 0.9854 - val_accuracy: 0.5263
Epoch 2/10
 - 2s - loss: 0.9938 - accuracy: 0.5044 - val_loss: 0.9817 - val_accuracy: 0.5263
Epoch 3/10
 - 2s - loss: 0.9441 - accuracy: 0.5254 - val_loss: 0.9536 - val_accuracy: 0.5877
Epoch 4/10
 - 2s - loss: 0.8647 - accuracy: 0.6102 - val_loss: 0.9844 - val_accuracy: 0.5614
Epoch 5/10
 - 2s - loss: 0.7420 - accuracy: 0.6796 - val_loss: 0.9319 - val_accuracy: 0.6053
Epoch 6/10
 - 2s - loss: 0.6009 - accuracy: 0.7424 - val_loss: 1.0238 - val_accuracy: 0.6228
Epoch 7/10
 - 2s - loss: 0.5196 - accuracy: 0.7951 - val_loss: 1.0079 - val_accuracy: 0.6316
Epoch 8/10
 - 2s - loss: 0.4162 - accuracy: 0.8356 - val_loss: 1.0201 - val_accuracy: 0.6491
Epoch 9/10
 - 2s - loss: 0.3178 - accuracy: 0.8817 - val_loss: 1.1666 - val_accuracy: 0.5789
Epoch 10/10
 - 2s - loss: 0.2298 - accuracy: 0.9264 - val_loss: 1.4004 - val_accuracy: 0.5351


In [239]:
gru_hist = gru_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train GRU with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 5s - loss: 1.0692 - accuracy: 0.4457 - val_loss: 1.0016 - val_accuracy: 0.5175
Epoch 2/10
 - 2s - loss: 1.0061 - accuracy: 0.5068 - val_loss: 0.9768 - val_accuracy: 0.5175
Epoch 3/10
 - 2s - loss: 0.9626 - accuracy: 0.5217 - val_loss: 0.9575 - val_accuracy: 0.5702
Epoch 4/10
 - 2s - loss: 0.8992 - accuracy: 0.5762 - val_loss: 0.9310 - val_accuracy: 0.5702
Epoch 5/10
 - 2s - loss: 0.8053 - accuracy: 0.6418 - val_loss: 0.9071 - val_accuracy: 0.5877
Epoch 6/10
 - 2s - loss: 0.6994 - accuracy: 0.6917 - val_loss: 0.8784 - val_accuracy: 0.6053
Epoch 7/10
 - 2s - loss: 0.5926 - accuracy: 0.7308 - val_loss: 0.9314 - val_accuracy: 0.6316
Epoch 8/10
 - 2s - loss: 0.4931 - accuracy: 0.7713 - val_loss: 0.9479 - val_accuracy: 0.6053
Epoch 9/10
 - 2s - loss: 0.4038 - accuracy: 0.8235 - val_loss: 0.9980 - val_accuracy: 0.6491
Epoch 10/10
 - 2s - loss: 0.3221 - accuracy: 0.8635 - val_loss: 1.1786 - val_accuracy: 0.6140


In [240]:
rnn_hist = rnn_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train RNN with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 3s - loss: 1.0450 - accuracy: 0.4793 - val_loss: 0.9855 - val_accuracy: 0.5263
Epoch 2/10
 - 1s - loss: 1.0004 - accuracy: 0.5054 - val_loss: 0.9738 - val_accuracy: 0.5263
Epoch 3/10
 - 1s - loss: 0.9625 - accuracy: 0.5086 - val_loss: 0.9590 - val_accuracy: 0.5175
Epoch 4/10
 - 1s - loss: 0.9085 - accuracy: 0.5319 - val_loss: 0.9586 - val_accuracy: 0.5175
Epoch 5/10
 - 1s - loss: 0.8093 - accuracy: 0.6339 - val_loss: 0.9463 - val_accuracy: 0.4649
Epoch 6/10
 - 1s - loss: 0.7078 - accuracy: 0.7205 - val_loss: 0.9537 - val_accuracy: 0.5526
Epoch 7/10
 - 1s - loss: 0.5674 - accuracy: 0.7844 - val_loss: 1.0625 - val_accuracy: 0.4561
Epoch 8/10
 - 1s - loss: 0.4613 - accuracy: 0.8491 - val_loss: 1.0502 - val_accuracy: 0.5351
Epoch 9/10
 - 1s - loss: 0.3526 - accuracy: 0.8919 - val_loss: 0.9856 - val_accuracy: 0.5351
Epoch 10/10
 - 1s - loss: 0.2684 - accuracy: 0.9255 - val_loss: 1.0727 - val_accuracy: 0.4912


In [241]:
from sklearn.metrics import classification_report

In [242]:
y_t=[]
for i in y_test:
  y_t.append(np.argmax(i))    #Using argmax to find the index of one hot encoding

In [243]:
y__pred =model.predict(x_test)      #predict y_test by bi-lstm with attention
y__p=[]
for i in y__pred:
  y__p.append(np.argmax(i))

In [244]:
print('Model = CNN\n')  
print(classification_report(y_t, y__p))

Model = CNN

              precision    recall  f1-score   support

           0       0.61      0.90      0.72       124
           1       0.67      0.06      0.10        71
           2       0.74      0.77      0.76        93

    accuracy                           0.65       288
   macro avg       0.67      0.58      0.53       288
weighted avg       0.66      0.65      0.58       288



In [245]:
y_ls_pred = ls_model.predict(x_test)      #predict y_test by bi-lstm with attention
y_ls_p=[]
for i in y_ls_pred:
  y_ls_p.append(np.argmax(i))


In [246]:
print('Model = Attention Bidirectonal LSTM\n')  
print(classification_report(y_t, y_ls_p))

Model = Attention Bidirectonal LSTM

              precision    recall  f1-score   support

           0       0.62      0.78      0.69       124
           1       0.48      0.14      0.22        71
           2       0.65      0.77      0.71        93

    accuracy                           0.62       288
   macro avg       0.58      0.57      0.54       288
weighted avg       0.59      0.62      0.58       288



In [247]:
y_sls_pred = sls_model.predict(x_test)  #predict y_test by lstm with attention
y_sls_p=[]
for i in y_sls_pred:
  y_sls_p.append(np.argmax(i))

In [248]:
print('Model = Attention LSTM\n')
print(classification_report(y_t, y_sls_p))

Model = Attention LSTM

              precision    recall  f1-score   support

           0       0.67      0.56      0.61       124
           1       0.36      0.38      0.37        71
           2       0.67      0.80      0.73        93

    accuracy                           0.59       288
   macro avg       0.57      0.58      0.57       288
weighted avg       0.59      0.59      0.59       288



In [249]:
y_gru_pred = gru_model.predict(x_test)    #predict y_test by GRU with attention
y_gru_p=[]
for i in y_gru_pred:
  y_gru_p.append(np.argmax(i))

In [250]:
print('Model = Attention GRU\n')
print(classification_report(y_t, y_gru_p))

Model = Attention GRU

              precision    recall  f1-score   support

           0       0.69      0.87      0.77       124
           1       1.00      0.06      0.11        71
           2       0.62      0.85      0.71        93

    accuracy                           0.66       288
   macro avg       0.77      0.59      0.53       288
weighted avg       0.74      0.66      0.59       288



In [251]:
y_rnn_pred = rnn_model.predict(x_test)    #predict y_test by RNN with attention
y_rnn_p=[]
for i in y_rnn_pred:
  y_rnn_p.append(np.argmax(i))

In [252]:
print('Model = Attention SimpleRNN\n')
print(classification_report(y_t, y_rnn_p))

Model = Attention SimpleRNN

              precision    recall  f1-score   support

           0       0.53      0.69      0.60       124
           1       0.30      0.20      0.24        71
           2       0.64      0.57      0.60        93

    accuracy                           0.53       288
   macro avg       0.49      0.48      0.48       288
weighted avg       0.51      0.53      0.51       288

